# Tutorial 5
Modal analysis - Free vibration

Jim Brouzoulis

In [3]:
import sys
import os
# Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
        
from mha021 import *

## CST element without and with supports

### Without

In [4]:
nodes = np.array([
    [0, 0], 
    [1, 0], 
    [0, 2]
])
D = hooke_2d_plane_stress(E=1, nu=0.3)
t = 1 # thickness

K, f = cst_element(nodes, D, t)
displayvar("K", K, accuracy=4)

M = cst_element_M(nodes, rho=1, t=t)
displayvar("M", M, accuracy=4)

omega2, phi = eigh(K, M)
# displayvar("ω^2", np.round(omega2), accuracy=4)
displayvar("f", np.sqrt(np.round(omega2))/(2*np.pi), accuracy=4)
displayvar("φ", phi, accuracy=4)

# Define elements and edofs so we can use the helper function plot_deformed_mesh
elements = np.array([
    [1, 2, 3]
])
edofs = np.array([
    [1, 2, 3, 4, 5, 6]
])


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### With supports

In [5]:
free_dofs = [5, 6] # removed 1, 2, 4
K_red = extract_block(K, free_dofs)
M_red = extract_block(M, free_dofs)
displayvar("K_{red}", K_red, accuracy=3)
displayvar("M_{red}", M_red, accuracy=3)
omega2, phi = eigh(K_red, M_red)

displayvar("ω^2", np.round(omega2), accuracy=4)
displayvar("f", np.sqrt(np.round(omega2))/(2*np.pi), accuracy=4)
displayvar("φ_{red}", phi, accuracy=4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
mode = 2
phi_mode = phi[:, mode-1]
num_dofs = 6
a = np.zeros((num_dofs)) 
a[np.array(free_dofs)-1] = phi_mode
displayvar("a", a, accuracy=4)


<IPython.core.display.Math object>

In [7]:

Ed = extract_dofs(a, edofs)
fig = plot_deformed_mesh(nodes, elements, Ed, scale=.2, field="utuxotal", title=f"Mode {mode}", cycles=3, duration=2)
fig.show()

### Mass spring system 

In [8]:
M = np.diag([0, 1, 1, 1, 1, 0])
displayvar("M", M)
K = np.zeros((6, 6))
ke = 1e3
assem(K, spring1e(ke), dofs=[1, 2])
assem(K, spring1e(ke), dofs=[2, 3])
assem(K, spring1e(ke), dofs=[3, 4])
assem(K, spring1e(ke), dofs=[4, 5])
assem(K, spring1e(ke), dofs=[5, 6])
displayvar("K", K)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
# Free vibration analysis
bc_dofs = [1, 6]
all_dofs = np.arange(1, num_dofs+1, 1, dtype=int)
free_dofs = np.setdiff1d(all_dofs, bc_dofs) 
# free_dofs = [2, 3, 4, 5]
K_red = extract_block(K, free_dofs)
M_red = extract_block(M, free_dofs)
displayvar("K_{red}", K_red)
displayvar("M_{red}", M_red)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:

omega2, phi_red = eigh(K_red, M_red)
displayvar("ω^2", omega2, accuracy=4)
displayvar("f", np.sqrt(omega2)/(2*np.pi), accuracy=4)
displayvar("φ_{red}", phi_red, accuracy=4)

phi_j = np.zeros((6))
phi_j[free_dofs-1] = phi_red[:, 0]
displayvar("φ_{j}", phi_j, accuracy=4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
K_phi = phi_red.T @ K_red @ phi_red
M_phi = phi_red.T @ M_red @ phi_red @ np.diag(omega2)
displayvar("K_\phi", np.round(K_phi), accuracy=4)
displayvar("M_\phi", np.round(M_phi), accuracy=4)

<>:3: SyntaxWarning:

invalid escape sequence '\p'

<>:4: SyntaxWarning:

invalid escape sequence '\p'

<>:3: SyntaxWarning:

invalid escape sequence '\p'

<>:4: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\mrmar\AppData\Local\Temp\ipykernel_3168\1255035562.py:3: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\mrmar\AppData\Local\Temp\ipykernel_3168\1255035562.py:4: SyntaxWarning:

invalid escape sequence '\p'



<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Modal analysis of truss 
See example from Tutorial 1 with a truss consisting of three elements.

This is exended to modal analysis

In [12]:
# Define input data
E = 210e9        # Young's modulus [Pa]
r = 0.02         # radius [m]
A = np.pi * r**2 # Cross-sectional area
Le = 1           # Element length [m]
P = 10e3         # Force [N]

Lx = Le * np.cos(np.radians(60))
Ly = Le * np.sin(np.radians(60))
Ex = np.array([
    [0, Lx], # el 1
    [1, Lx],
    [0, Le]
])
Ey = np.array([ 
    [0, Ly], # el 1
    [0, Ly],
    [0, 0]
])

# Topology matrix (connectivity)
Edof = np.array([
    [1, 2, 5, 6],   # Element 1
    [3, 4, 5, 6],   # Element 2
    [1, 2, 3, 4]    # Element 3
])
# Number of elements
num_el = 3
num_dofs = 6

# Assemble stiffness matrix and load vector, first allocate space
K = np.zeros((num_dofs, num_dofs)) # Stiffness matrix
M = np.zeros_like(K)               # Mass matrix
f = np.zeros((num_dofs))           # Load vector

def bar2m(rho, L, A):
    # return consistent mass matrix for a bar in 2D
    return rho*A*L/6 * np.array([
        [2, 0, 1, 0],
        [0, 2, 0, 1],
        [1, 0, 2, 0],
        [0, 1, 0, 2],
    ])

# Loop over all elements to assemble global stiffness matrix
rho = 7800
for el in range(num_el):
    Ke = bar2e(Ex[el, :], Ey[el, :], E = E, A = A)  # Element stiffness matrix
    Me = bar2m(rho, Le, A)
    dofs = Edof[el, :] 
    assem(K, Ke, dofs)
    assem(M, Me, dofs)

displayvar("K", np.round(K))
displayvar("M", np.round(M))

free_dofs = np.array([3, 5])
K_red = extract_block(K, free_dofs)
M_red = extract_block(M, free_dofs)

displayvar("K_{red}", np.round(K_red))
displayvar("M_{red}", np.round(M_red))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:

omega2, phi_red = eigh(K_red, M_red)
displayvar("ω²", omega2)
displayvar("\phi_{red}", phi_red)
# External forces
# f[5-1] = P  # Add a horizontal force at node 3 (= dof 5)

# # Boundary conditions
# bc_dofs = np.array([1, 2, 4, 6]) # DOFs fixed: 1, 2, 4, 6
# bc_vals = np.zeros_like(bc_dofs)

# # Solve the system of equations
# a, r = solve_eq(K, f, bc_dofs, bc_vals)
# displayvar("a", a, accuracy=4)
# displayvar("r", r, accuracy=4)

# Ed = extract_dofs(a, Edof)
# fig = draw_discrete_elements(
#     Ex, Ey,
#     title="Deformed truss",
#     xlabel="x [m]",
#     ylabel="y [m]",
#     line_style="dashed"
# )
# plot_deformed_bars(fig, Ex, Ey, Ed, scale=1e3)
# fig.show()


<>:3: SyntaxWarning:

invalid escape sequence '\p'

<>:3: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\mrmar\AppData\Local\Temp\ipykernel_3168\1149539207.py:3: SyntaxWarning:

invalid escape sequence '\p'



<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Modal analysis

In [14]:
# Function for mass matrix
def bar2m(rho, L, A):
    # return consistent mass matrix for a bar in 2D
    return rho*A*L/6 * np.array([
        [2, 0, 1, 0],
        [0, 2, 0, 1],
        [1, 0, 2, 0],
        [0, 1, 0, 2],
    ])

# M = np.zeros_like(K)
# Me = bar2m(rho, Le, A)
# assem(M, Me, dofs)

# Solve for natural frequencies and modes
free_dofs = [3, 5, 6]
K_red = extract_block(K, free_dofs)
M_red = extract_block(M, free_dofs)
omega2, phi_red = eigh(K_red, M_red)

displayvar("\omega", np.sqrt(omega2), accuracy=4)
displayvar("\phi_{red}", phi_red, accuracy=4)



<>:21: SyntaxWarning:

invalid escape sequence '\o'

<>:22: SyntaxWarning:

invalid escape sequence '\p'

<>:21: SyntaxWarning:

invalid escape sequence '\o'

<>:22: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\mrmar\AppData\Local\Temp\ipykernel_3168\1493547602.py:21: SyntaxWarning:

invalid escape sequence '\o'

C:\Users\mrmar\AppData\Local\Temp\ipykernel_3168\1493547602.py:22: SyntaxWarning:

invalid escape sequence '\p'



<IPython.core.display.Math object>

<IPython.core.display.Math object>

### plot vibration mode

In [15]:

# "Determine a" first create an empty array and the fill it with data from phi_red  
phi_j = np.zeros((6)) # size = number of dofs

mode = 1 # mode to extract
phi_j[np.array(free_dofs)-1] = phi_red[:, mode-1] 
Ed = extract_dofs(phi_j, Edof)
fig = draw_discrete_elements(
    Ex, Ey,
    title=f"Mode shape {mode}",
    xlabel="x [m]",
    ylabel="y [m]",
    line_style="dashed"
)
plot_deformed_bars(fig, Ex, Ey, Ed, scale=1)
fig.show()
